In [3]:
import sys
import pandas as pd
import matplotlib
import numpy as np
import scipy as sp
import IPython
from IPython import display
import sklearn
import random
import time
import warnings
warnings.filterwarnings('ignore')
from subprocess import check_output

In [4]:
# common algorisms
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier
# model helpers

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics

#Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.tools.plotting import scatter_matrix

%matplotlib inline
mpl.style.use('ggplot')
sns.set_style('white')
pylab.rcParams['figure.figsize'] = 12,8


In [5]:
# read data
data_row= pd.read_csv('../input/train.csv')
data_val = pd.read_csv('../input/test.csv')
data1 = data_row.copy(deep = True)
data_cleaner = [data1, data_val]

In [6]:
print(data_row.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4459 entries, 0 to 4458
Columns: 4993 entries, ID to 9fc776466
dtypes: float64(1845), int64(3147), object(1)
memory usage: 169.9+ MB
None


In [7]:
data_row.sample(10)

,ID,target,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,...,3ecc09859,9281abeea,8675bec0b,3a13ed79a,f677d4d13,71b203550,137efaa80,fb36b89d9,7e293fbaf,9fc776466
4233,f27708db0,247000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
1091,3f13fba95,6000000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
1974,72aef8ea8,14200000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,404000.0,0,0,0,0,0,0,0
4021,e6762b97d,100000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
2171,7ebdd3164,2000000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,17000000,0,0,0,0,0,0
2179,7f5dc5e8d,4000000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
1493,568996783,500000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
1016,3a9d2ab52,10000000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
4321,f72b1dd9b,1222000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
1753,66d1b1e1e,30000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,254000,0


In [8]:
data_val.sample(10)

,ID,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,20aa07010,...,3ecc09859,9281abeea,8675bec0b,3a13ed79a,f677d4d13,71b203550,137efaa80,fb36b89d9,7e293fbaf,9fc776466
47335,f56a2df8c,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
48846,fd579753f,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2479,0ccd16f73,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32494,a8305109d,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8408,2aea5238d,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37620,c32fa09f4,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,3761200.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36728,beafd6ddd,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7446,26159b17e,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,6535000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19399,6356296a5,0.0,0.0,0.0,0.0,2.375205e+06,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
47114,f44b4fb47,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


顧客の潜在的な価値を分析する

テストデータには、targetのみがない。

targetの値を予測する

In [9]:
# 値が全て0のカラムを探す
data1_sum = data1.isin([0]).sum()
data1_sum

ID              0
target          0
48df886f9    4420
0deb4b6a8    4455
34b15f335    4426
a8cb14b00    4457
2f0771a37    4454
30347e683    4412
d08d1fbe3    4454
6ee66e115    4454
20aa07010    2965
dc5a8f1d8    4336
11d86fa6a    4294
77c9823f2    4458
8d6c2a0b2    4453
4681de4fd    3940
adf119b9a    4299
cff75dd09    4424
96f83a237    4434
b8a716ebf    4125
6c7a4567c    4431
4fcfd2b4d    4432
f3b9c0b95    4420
71cebf11c    4438
d966ac62c    4380
68b647452    4328
c88d108c9    4448
ff7b471cd    4455
d5308d8bc    4459
0d866c3d7    4203
             ... 
cdfc2b069    3816
2a879b4f7    4241
6b119d8ce    4075
98dea9e42    4455
9f2471031    4458
88458cb21    4446
f40da20f4    4440
7ad6b38bd    4357
c901e7df1    4284
8f55955dc    4450
85dcc913d    4370
5ca0b9b0c    4458
eab8abf7a    4326
8d8bffbae    4002
2a1f6c7f9    4426
9437d8b64    4127
5831f4c76    3747
2e84e09c5    4458
d45fd5508    4427
a165f5761    4353
3ecc09859    4168
9281abeea    4274
8675bec0b    3766
3a13ed79a    4294
f677d4d13 

In [30]:
data1_mask = list(data1_sum[data1_sum == 4458].index)

In [31]:
data1_mask

['77c9823f2',
 'e90ed19da',
 'cde9c35e8',
 '604ac0633',
 '1de1fda2c',
 '5d26f4d92',
 '4a3248e89',
 'd6033b6eb',
 'bf6f90180',
 'fcab1682f',
 'cc35ec717',
 '5df2068b0',
 '29714c47d',
 '34d3974de',
 '04e06920e',
 '3ad3d521e',
 'bacadce94',
 '7497a6bc9',
 'bfde2aa61',
 'f519ac616',
 '7d72d6787',
 '9abaeaeba',
 '37e85aa12',
 '995a96a47',
 '3c29aec1e',
 'e7fa14f98',
 'e5355e47e',
 'e851264a5',
 'c6d27ddbb',
 '279f03bdc',
 'b07303fc6',
 '403b0e1e4',
 'd74a5ca6d',
 'a47434e94',
 '3c556d78f',
 'dea4edd89',
 '78f7fcebd',
 '871617f50',
 'e83d8d925',
 '969d32625',
 '1d0affea2',
 '8405c17e7',
 '1655bb8bb',
 'a028a97a2',
 '0c8e0cd38',
 'b54c15cae',
 '30cef4483',
 '50c81fe9f',
 'aa2e796b4',
 'acd155589',
 'b9e9ec060',
 '40ad014d1',
 '925ce0d9b',
 '50722cac3',
 '170b48e6b',
 'c84209935',
 'ecd43a18b',
 '4fcb73cb1',
 '939cb6e17',
 '1ab7a3c7f',
 '70f3a87ec',
 '66f57f2e5',
 '4c256f2f9',
 'a5dfa7a30',
 'c6776639f',
 'dcc181073',
 '22b3e64c8',
 'f9a30f20a',
 'f66c58ca7',
 '598ae7ea9',
 'ddc07c4d6',
 '2306

data1.columns

このデータはカラムにラベルがついてるので、axis=1

In [39]:
data1.drop(data1_mask, axis=1, inplace=True)

In [40]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4459 entries, 0 to 4458
Columns: 4760 entries, ID to 9fc776466
dtypes: float64(1845), int64(2914), object(1)
memory usage: 161.9+ MB


In [41]:
data_val_sum = data_val.isin([0]).sum()

In [42]:
data_val_sum

ID               0
48df886f9    49056
0deb4b6a8    49146
34b15f335    48810
a8cb14b00    49138
2f0771a37    49162
30347e683    49032
d08d1fbe3    49128
6ee66e115    49152
20aa07010    41878
dc5a8f1d8    48956
11d86fa6a    48974
77c9823f2    49085
8d6c2a0b2    49092
4681de4fd    46380
adf119b9a    48674
cff75dd09    48825
96f83a237    49140
b8a716ebf    48469
6c7a4567c    48812
4fcfd2b4d    49144
f3b9c0b95    48944
71cebf11c    49111
d966ac62c    48300
68b647452    48597
c88d108c9    49081
ff7b471cd    49150
d5308d8bc    49034
0d866c3d7    48477
bc3f77679    49097
             ...  
cdfc2b069    46320
2a879b4f7    48407
6b119d8ce    47779
98dea9e42    49106
9f2471031    49114
88458cb21    49066
f40da20f4    49131
7ad6b38bd    48572
c901e7df1    48253
8f55955dc    49130
85dcc913d    48768
5ca0b9b0c    49143
eab8abf7a    48570
8d8bffbae    48587
2a1f6c7f9    48897
9437d8b64    48063
5831f4c76    45934
2e84e09c5    49113
d45fd5508    48927
a165f5761    48924
3ecc09859    48604
9281abeea   

In [ ]:
data_val.info()